In [76]:
import pandas as pd
import numpy as np
from icecream import ic
from typing import List


In [77]:
MAIN_PHARMAWATCH_DATA = "../data/pharmawatch.csv"
ENRICHED_MATERIAL_DATA = "../data/enriched_material.csv"

In [78]:
df_pharmawatch = pd.read_csv(MAIN_PHARMAWATCH_DATA, sep=",", nrows=100)
df_pharmawatch.head()

,Material,Unidade,Quantidade,Valor unitário (R$),Valor total (R$),Mês,Ano,Local,Destino
0,ÁCIDO ACETIL SALICILICO 100 MG,UNIDADE,"3,00","0,08","0,24",1,2021,FARMÁCIA CENTRO CIRÚRGICO (SATÉLITE),BLOCO CIRÚRGICO
1,ÁCIDO EPSILON AMINOCAPROICO 1G/20ML,UNIDADE,"63,00","23,47","1.478,61",1,2021,FARMÁCIA CENTRO CIRÚRGICO (SATÉLITE),BLOCO CIRÚRGICO
2,ÁCIDO EPSILON AMINOCAPROICO 4G/20ML,UNIDADE,"26,00","27,90","725,40",1,2021,FARMÁCIA CENTRO CIRÚRGICO (SATÉLITE),BLOCO CIRÚRGICO
3,ÁCIDO TRANEXAMICO 50 MG/ML INJETÁVEL,UNIDADE,"61,00","4,59","279,99",1,2021,FARMÁCIA CENTRO CIRÚRGICO (SATÉLITE),BLOCO CIRÚRGICO
4,AGUA DESTILADA 10ML,UNIDADE,"600,00","0,28","168,00",1,2021,FARMÁCIA CENTRO CIRÚRGICO (SATÉLITE),BLOCO CIRÚRGICO


In [79]:
df_enriched_material = pd.read_csv(ENRICHED_MATERIAL_DATA)
df_enriched_material.head()

,Material,Apresentação farmacêutica,Via de administração,Antibiótico (Sim/Não),Tipo,Subtipo,Origem do recurso
0,"ACETATO DE LEUPRORRELINA 22,5MG",Injetáveis,Injetável,Não,Não Antibiótico,NaN,Próprio hospital
1,"ACETATO DE LEUPRORRELINA 7,5MG",Injetáveis,Injetável,Não,Não Antibiótico,NaN,Próprio hospital
2,ACETATO DE RETINOL AMINOÁCIDO-METIONINA-CLORAN...,Pomada,Tópica,Não,Não Antibiótico,NaN,Próprio hospital
3,ACETAZOLAMIDA 250 MG,Comprimido,Oral,Não,Não Antibiótico,NaN,Próprio hospital
4,ACICLOVIR 200 MG COMPRIMIDO,Comprimido,Oral,Não,Não Antibiótico,NaN,Próprio hospital


In [80]:
enrich_data = {
    "Material": df_enriched_material
}

words_to_remove = [
    "POMADA",
]

floating_pointers_to_adjust = [
    'Quantidade',
    'Valor unitário (R$)',
    'Valor total (R$)'
]

In [90]:
def adjust_number_floating_pointers(df: pd.DataFrame, columns: List[str]):
    for column in columns:
        df[column] = df[column].apply(lambda x: x.replace(".", ""))
        df[column] = df[column].apply(lambda x: x.replace(",", "."))
        df[column] = df[column].astype(float)
    return df

def group_by_material(df: pd.DataFrame):
    df = df.groupby(["Material", "Unidade", "Mês", "Ano", "Local", "Destino", "Valor unitário (R$)"]).agg({"Quantidade": "sum", "Valor total (R$)": "sum"}).reset_index()
    return df

def split_material_components(df: pd.DataFrame):
    """ 
    df has a column named 'material', i need to break it in tree columns: material_base, material_comp and metod_administracao
    material_base is the first part until the first "+" or before any set of numbers that does not finish with "%"
    material_comp is the optional part after the first "+" and before any set of numbers that does not finish with "%"
    metod_administracao is the final part from any set of numbers that does not finish with "%" until the end of the string
    """
    df['Material Base'] = df['Material'].str.extract(r'^(.*?)(?:\+|\d+[^%])')
    df['Material Comp'] = df['Material'].str.extract(r'(?<=\+)(.*?)(?:\d+[^%])')
    df['Metod Administracao'] = df['Material'].str.extract(r'(\d+[^%]*$)')
    df[['Material Base', 'Material Comp', 'Metod Administracao']] = df[['Material Base', 'Material Comp', 'Metod Administracao']].apply(lambda x: x.str.strip())
    return df

def clean_material_components(df: pd.DataFrame, list_words_to_remove: List[str]):
    for word in list_words_to_remove:
        df['Material Base'] = df['Material Base'].str.replace(word, '')
        df['Material Comp'] = df['Material Comp'].str.replace(word, '')
    return df

def enrich_dataframe(df: pd.DataFrame, **enriched_data):
    for column, df_enriched in enriched_data["enriched_data"].items():
        if "Subtipo" in df_enriched.columns:
            df_enriched = df_enriched.drop(columns=['Subtipo'])
        if "Antibiótico (Sim/Não)" in df_enriched.columns:
            df_enriched = df_enriched.drop(columns=['Antibiótico (Sim/Não)'])
        df = df.merge(df_enriched, on=column, how='left')
    return df

In [91]:
def run(df):
    df = df.copy()
    df = adjust_number_floating_pointers(df, floating_pointers_to_adjust)
    df = group_by_material(df)
    df = split_material_components(df)
    df = clean_material_components(df, words_to_remove)
    df = enrich_dataframe(df, enriched_data=enrich_data)
    return df

In [92]:
df_enriched_pharmawatch = run(df_pharmawatch)
df_enriched_pharmawatch.head()

,Material,Unidade,Mês,Ano,Local,Destino,Valor unitário (R$),Quantidade,Valor total (R$),material_base,material_comp,metod_administracao,Apresentação farmacêutica,Via de administração,Tipo,Origem do recurso
0,AGUA DESTILADA 10ML,UNIDADE,1,2021,FARMÁCIA CENTRO CIRÚRGICO (SATÉLITE),BLOCO CIRÚRGICO,0.28,600.0,168.00,AGUA DESTILADA,NaN,10ML,Ampola,Injetável,Material,Próprio hospital
1,AGUA OXIGENADA 10 VOLUMES FR 1000 ML,FRASCO,1,2021,FARMÁCIA CENTRO CIRÚRGICO (SATÉLITE),BLOCO CIRÚRGICO,8.00,15.0,120.00,AGUA OXIGENADA,NaN,10 VOLUMES FR 1000 ML,Frasco,Tópica,Não Antibiótico,Próprio hospital
2,ALBUMINA HUMANA 200 MG/ML 50 ML,UNIDADE,1,2021,FARMÁCIA CENTRO CIRÚRGICO (SATÉLITE),BLOCO CIRÚRGICO,120.00,6.0,720.00,ALBUMINA HUMANA,NaN,200 MG/ML 50 ML,Solução,Injetável,Não Antibiótico,Próprio hospital
3,AMIODARONA 50MG/ML AMPOLA 3ML,UNIDADE,1,2021,FARMÁCIA CENTRO CIRÚRGICO (SATÉLITE),BLOCO CIRÚRGICO,2.15,4.0,8.60,AMIODARONA,NaN,50MG/ML AMPOLA 3ML,Ampola,Injetável,Não Antibiótico,Próprio hospital
4,"ATRACURIO 10 MG/ML 2,5 ML",UNIDADE,1,2021,FARMÁCIA CENTRO CIRÚRGICO (SATÉLITE),BLOCO CIRÚRGICO,24.66,3.0,73.98,ATRACURIO,NaN,"10 MG/ML 2,5 ML",Ampola,Injetável,Não Antibiótico,Próprio hospital
